#### Scenario

1. 기준값적용 (평가기준값)
 - 기준값 지정 (총 5점)
 - 평가값을 받는 것 = 
이동수단 ~ 거리 ** 출발지점기준~관광지거리 / 
체류기간~비용/
개인만족도
x) 기준 값 이동수단 도보5점 / 버스 4점 /자동차  3점 / 전철 2점 / 비행기 1점)
               비용 (5만원 5점 / 10만원 4점 / 15만원 3점 / 20만원 2점 / 20만원 이상 1점)
               거리 (서울역: 좌표 기준 -> 해당 관광지 좌표 인풋시 직선거리 계산)
                     15km 간격으로 5 4 3 2 1점 계산 (from haversine import haversine)


거리에 대한 이동수단 평가 --> 구글맵..위경도설정 이거 가능한거 맞냐 안될 것 같은데

도보 
거리 1km 미만 - 5점
거리 2km - 4점
거리 3km - 3점
거리 4km - 2점
거리 5km 이상- 1점

지역버스
거리 5km 미만 - 5점
거리 15km - 4점
거리 25km - 3점
거리 35km - 2점
거리 45km이상 -1점

자차
거리 10km 미만 -5점
거리 30km 4점
거리 50km 3점
거리 70km 2점
거리 90km 1점

고속버스
거리 20km 미만 -5점
거리 50km 4점
거리 80km 3점
거리 110km 2점
거리 140km 1점

전철
거리 40km 미만 -5점
거리 70km 4점
거리 100km 3점
거리 130km 2점
거리 160km 1점


#체류기간에 대한 비용값 평가 (비용->1일 체류기간에 대한 평균 지출금액)
(평균 비용 일 133,000원 관광데이터랩 1회 평균 국내여행지출)
133,000원 평균 값 = 5점 만점

133,000*2.1%이하 5점
133,000*2.1%초과 133,000*13.6%이하 4점 
133,000*13.6% 초과 133,000*50%이하 3점
133.000*50% 초과 133,000*84.1% 이하 3점
133,000*84.1%초과 133,000*97.7%이하 2점 
133,000*97.7%이상 1점

--> 평균값 133,000원 정확한 범위 분배해서 1~5점 설정.

#개인만족도 (5점 만점)



========================================================
기록
1. 관광지 날짜/이름/타입\
관광지 인덱스 내용 /이동수단/비용/거리(*계산식 고민)
/다녀온 관광지(*데이터 int,count값)
2. 데이터 세트를 각 txt파일에 저장.
3. txt파일 누적으로 저장
4. 파일은 다음과 같음
일자 이름  타입 이동수단 (평가) 비용(평가) 거리(평가)   
06   서울  도시   자차             만원        3점  

평가
 인풋된 데이터와 비교
https://mizykk.tistory.com/21
2. 기준 인풋 데이터를 토대로(평가) 관광지에 대한 평가 평균값을 계산.
3. Top 레이팅을 평가.
------------------------------------------
탐색
1. 이름/탐색 기준으로 key 값 지정
2. 평균값 내림차순으로 Key값에 의한 sort
3. 원하는 지역 탐색시 도메인 + 쿼리에 검색내용 추가 : 링크로 이동할 수 있게끔 도와줌
4. 

              

###### 1. Program begins

###### 2. Program data set

2-1 Data record

In [ ]:
class Record:
    
    def record_tour_data(self):
        
        while True:
            Question = input("처음 기록하십니까? Y/N : ")
            
            
            if Question == "Y":
                while True:
                p_date = int(input("여행날짜 : ")) #datetime 정규식으로 나눌것
                p_name = input("여행목적지(관광지이름) : ")
                p_type = input("관광지유형을 고르세요. 도시/바다/캠핑/등")
                p_start = input("출발 장소(도로명주소) 를 입력하세요 : ")
                p_destination = input("중요 관광지(도로명주소)를 입력하세요 : ")
                p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
                p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                p_cost = int(input("총 지출 금액 : __ 원"))
                past_data =[p_date,p_name,p_type,p_start,p_transportation,p_length_of_stay,p_cost]
                past_data =[p_cost]
                past_data = str(past_data)
                
                with open ("past_data.txt","w") as p_d:
                        p_d.write(past_data)
                
                Question_end = input("다른 여행지를 기록하시겠습니까? Y/N : ")
                if Question_end == "Y":
                    with open("past_data.txt","a") as p_da:
                        p_da.write(past_data)
                
            
            
    
            
            
        
 # While       

2-2. Data Calucation sp)L_S_D

In [ ]:
class Distance:
    def getLatLng(self):
        #모듈 다운로드
        !pip install haversine
        #출발장소 도로명주소 위/경도로 변환 --float값으로 변환
        addr_1 = input("출발 장소(도로명주소) 를 입력하세요 : ")
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_1
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        first_destination_Lat = float(match_first['y'])
        first_destination_Lng = float(match_first['x'])
        first_LatLng = (first_destination_Lat,first_destination_Lng)

        addr_2 = input("여행 장소(도로명주소) 를 입력하세요 : ")
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_2
        headers = {"Authorization" : "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_second = result['documents'][0]['address']
        second_destination_Lat = float(match_second['y'])
        second_destination_Lng = float(match_second['x'])
        second_LatLng = (second_destination_Lat,second_destination_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(first_LatLng,second_LatLng))
        print(type(r_s_d))
        return r_s_d


In [157]:
#Dummy
getLatLng()

출발 장소(도로명주소) 를 입력하세요 : 걸포1로 10
여행 장소(도로명주소) 를 입력하세요 : 만리재로 175
<class 'int'>


24

###### 주소 가져오기 
간단한 명칭만 넣으면 주소를 가져옴

In [170]:
import requests

searching = '경기도 김포시 걸포1로 10'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
headers = {
    "Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"
}
places = requests.get(url, headers = headers).json()['documents']
places[0]


KeyError: 'documents'

2-2. Data Standard "Evauation"

In [150]:
#  기준값적용 (평가기준값)
#  - 기준값 지정 (총 5점)
#  - 평가값을 받는 것 = 이동수단 / 비용 / 거리 ** 서울역 기준 /   

class evaluation:
    
    def evalue_costs(self,p_cost): #관광데이터랩 1회 평균 국내여행지출 금액 133,000원 정규분포도 활용
        if p_cost/p_length_of_stay <= 133000*0.5: #665000
            return 5
        elif 133000*0.5 < p_cost/p_length_of_stay <= 133000*0.8: #665000~106400
            return 4
        elif 133000*0.8 < p_cost/p_length_of_stay <= 133000*1.1: #106400  #~ 1463000
            return 3
        elif 133000*1.1 < p_cost/p_length_of_stay <= 133000*1.3: #146300 ~ 172900
            return 2
        else: #172900
            return 1
        
    def evalue_by_distance(self): #이동수단을 직선거리로 평가.
        if p_transportation == "도보":
            if r_s_d < 1:
                return 5
            elif r_s_d < 2:
                return 4
            elif r_s_d < 3:
                return 3
            elif r_s_d < 4:
                return 2
            else:
                return 1
            
        elif p_transportation == "지역버스":
            if r_s_d < 5:
                return 5
            elif r_s_d < 15:
                return 4
            elif r_s_d < 25:
                return 3
            elif r_s_d < 35:
                return 2
            else:
                return 1
            
        elif p_transportation == "자동차":
            if r_s_d < 10:
                return 5
            elif r_s_d < 30:
                return 4
            elif r_s_d < 50:
                return 3
            elif r_s_d < 70:
                return 2
            else:
                return 1
            
        elif p_transportation == "고속버스":
            if r_s_d < 20:
                return 5
            elif r_s_d < 50:
                return 4
            elif r_s_d < 80:
                return 3
            elif r_s_d < 110:
                return 2
            else:
                return 1
            
        elif p_transportation == "전철":
            if r_s_d < 40:
                return 5
            elif r_s_d < 70:
                return 4
            elif r_s_d < 100:
                return 3
            elif r_s_d < 130:
                return 2
            else:
                return 1
        
    def evalue_by_satisfaction(self): #관광만족도 개인 평가 
        if p_satisfaction <= 1: #665000
            return 1
        elif 1 < p_satisfaction <= 2: #665000~106400
            return 2
        elif 2 < p_satisfaction <= 3: #106400  #~ 1463000
            return 3
        elif 3 < p_satisfaction <= 4: #146300 ~ 172900
            return 4
        else: #172900
            return 5
        
    
            
        

2-2. Record

In [57]:
class Record:
    def record_tour_data(self):
        Question = input("처음 기록하십니까? Y/N")
        if Question == Y:
            p_date = int(input("여행날짜 : ")) #datetime 정규식으로 나눌것
            p_name = input("여행목적지(관광지이름) : ")
            p_type = input("관광지유형을 고르세요. 도시/바다/캠핑/등")
            p_start = input("출발지 : ")
            p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
            p_length_of_stay = int(input("총 여행 기간 : __ 일"))
            p_cost = int(input("총 지출 금액 : __ 원"))
            past_data =[p_date,p_name,p_type,p_start,p_transportation,p_length_of_stay,p_cost]
            past_data = str(past_data)
            with open ("past_data.txt",'w') as p_d:
                p_d.write(past_data)
            
        else:
            Answer_Q = print("다음 여행을 기록합니다.")
            
            

        
 # While       

In [37]:
class Record:
    
    def record_tour_data(self):
        
        while True:
            Question = input("처음 기록하십니까? Y/N : ")
            
            
            if Question == "Y":
                while True:
#                 p_date = int(input("여행날짜 : ")) #datetime 정규식으로 나눌것
#                 p_name = input("여행목적지(관광지이름) : ")
#                 p_type = input("관광지유형을 고르세요. 도시/바다/캠핑/등")
#                 p_start = input("출발지 : ")
#                 p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
#                 p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                    p_cost = int(input("총 지출 금액 : __ 원"))
#                 past_data =[p_date,p_name,p_type,p_start,p_transportation,p_length_of_stay,p_cost]
                    past_data =[p_cost]
                    past_data = str(past_data)
                
                    with open ("past_data.txt","w") as p_d:
                        p_d.write(past_data)
                
                Question_end = input("다른 여행지를 기록하시겠습니까? Y/N : ")
                if Question_end == "Y":
                    with open("past_data.txt","a") as p_da:
                        p_da.write(past_data)
                
            
            
    
            
            
        
 # While       

In [ ]:
Dummy = Record()
Dummy.record_tour_data()

처음 기록하십니까? Y/N : Y
총 지출 금액 : __ 원1
총 지출 금액 : __ 원2
총 지출 금액 : __ 원3


###### 위도경도 차이 구하기

In [151]:
!pip install haversine
from haversine import haversine
k = (37.504030,127.024099)
g = (37.497175,127.027926)
print(type(k[0]))
print(int(haversine(k,g)*1000),'km')

<class 'float'>
833 km


In [210]:
# def getLatLng():
    import requests
    p_start = input("출발 장소(도로명주소) 를 입력하세요 : ")
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"
    }
    places = requests.get(url, headers = headers).json()['documents']
    places[0]
    #address_name
    #

IndentationError: unexpected indent (<ipython-input-210-ea8990dfa4a7>, line 2)

In [239]:
def getLatLng():
    import requests
    p_start = input("출발 장소(도로명주소) 를 입력하세요 : ")
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
    headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
    p_s_places = requests.get(url, headers = headers).json()['documents']
    p_s_Lat = float(p_s_places[0]['x'])
    p_s_Lng = float(p_s_places[0]['y'])
    p_s_LatLng=(p_s_Lat,p_s_Lng)
    
    p_destination = input("중요 관광지(도로명주소)를 입력하세요 : ")
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_destination)
    headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
    p_d_places = requests.get(url, headers = headers).json()['documents']
    p_d_Lat = float(p_d_places[0]['x'])
    p_d_Lng = float(p_d_places[0]['y'])
    p_d_LatLng=(p_d_Lat,p_d_Lng)
    
    #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
    #상대적 직선거리 (relative straight distance 변수에 지정)
    from haversine import haversine
    r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
    print(type(r_s_d))
    return r_s_d
    
    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

In [241]:
getLatLng()

출발 장소(도로명주소) 를 입력하세요 : 걸포1로 10
중요 관광지(도로명주소)를 입력하세요 : 서울역
<class 'int'>


29

In [196]:
class Distance:
    def getLatLng(self):
        #모듈 다운로드
#         !pip install haversine
        #출발장소 도로명주소 위/경도로 변환 --float값으로 변환
        addr_1 = input("출발 장소(도로명주소) 를 입력하세요 : ")
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_1
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        result = json.loads(str(requests.get(url, headers=headers).text))
#         match_first = result['documents'][0]['address']
#         first_destination_Lat = float(match_first['y'])
#         first_destination_Lng = float(match_first['x'])
#         first_LatLng = (first_destination_Lat,first_destination_Lng)
        print(result)

#         addr_2 = input("여행 장소(도로명주소) 를 입력하세요 : ")
#         url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_2
#         headers = {"Authorization" : "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
#         result = json.loads(str(requests.get(url, headers=headers).text))
#         match_second = result['documents'][0]['address']
#         second_destination_Lat = float(match_second['y'])
#         second_destination_Lng = float(match_second['x'])
#         second_LatLng = (second_destination_Lat,second_destination_Lng)

#         #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
#         #상대적 직선거리 (relative straight distance 변수에 지정)
#         from haversine import haversine
#         r_s_d = int(haversine(first_LatLng,second_LatLng))
#         print(type(r_s_d))
#         return r_s_d


In [198]:
A = Distance()
A.getLatLng()

출발 장소(도로명주소) 를 입력하세요 : 김포 현대 프리미엄 아울렛
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}


In [209]:
import requests

searching = '경기도 김포시 걸포1로 10'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
headers = {
    "Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"
}
places = requests.get(url, headers = headers).json()['documents']
places[0]
#address_name
#

{'address_name': '경기 김포시 걸포동 1561',
 'category_group_code': '',
 'category_group_name': '',
 'category_name': '부동산 > 주거시설 > 아파트',
 'distance': '',
 'id': '11327671',
 'phone': '',
 'place_name': '오스타파라곤2단지아파트',
 'place_url': 'http://place.map.kakao.com/11327671',
 'road_address_name': '경기 김포시 걸포1로 10',
 'x': '126.70562812359279',
 'y': '37.63344397411889'}

In [191]:
# from urllib.parse import quote_plus, urlencode
# from urllib.request import urlopen, Request
# import xml.etree.ElementTree as ET
# import json


# print('도로명주소 검색API 서비스를 이용한 주소검색결과를 보여줍니다.')
# keystr = input('검색어를 입력하세요. : ')
# resulttype = input('result type을 입력하세요(0 => xml, 1=> json) : ')

# if resulttype=='1':
#     resulttype= 'json'
# else:
#     resulttype = 'xml'

# #print(resulttype)

# url = 'http://www.juso.go.kr/addrlink/addrLinkApi.do'
# queryParams = '?' + urlencode({ quote_plus('currentPage') : '1' , 
#                                quote_plus('countPerPage') : '10', 
#                                quote_plus('resultType') : resulttype, 
#                                quote_plus('keyword') : keystr, 
#                                quote_plus('confmKey') : 'devU01TX0FVVEgyMDIxMDcwMTE3Mzg0NDExMTM0Nzk='})

# request = Request(url + queryParams)
# request.get_method = lambda: 'GET'
# response_body = urlopen(request).read()

# #print(response_body.decode('utf-8'))

# print("url = " + url)
# print("keyword = " + keystr)
# print("="*100)


# if resulttype=='json':
#     root_json = json.loads(response_body)
#     print('<< results >>')
#     print('totalCount   : ' + root_json['results']['common']['totalCount'])
#     print('currentPage  : ' + root_json['results']['common']['currentPage'])
#     print('countPerPage : ' + root_json['results']['common']['countPerPage'])
#     print('errorCode    : ' + root_json['results']['common']['errorCode'])
#     print('errorMessage : ' + root_json['results']['common']['errorMessage'])
#     for child in root_json['results']['juso']:
#         print('-'*100)
#         print('[' + child['zipNo'] + '] ' + child['roadAddr'])
#         print('    지번주소     = ' + child['jibunAddr'])
#         print('    영문주소     = ' + child['engAddr'])
#         print('    도로명코드   = ' + child['rnMgtSn'])
#         print('    건물관리번호 = ' + child['bdMgtSn'])
#         print('    법정동코드   = ' + child['admCd'])
#         print('    상세건물명   = ' + child['detBdNmList'])
#         print('')
    

# else:
#     root_xml = ET.fromstring(response_body)

#     print('<< ' + root_xml.tag + ' >>')

#     print('totalCount   : ' + root_xml.find('common').findtext('totalCount'))
#     print('currentPage  : ' + root_xml.find('common').findtext('currentPage'))
#     print('countPerPage : ' + root_xml.find('common').findtext('countPerPage'))
#     print('errorCode    : ' + root_xml.find('common').findtext('errorCode'))
#     print('errorMessage : ' + root_xml.find('common').findtext('errorMessage'))

#     for child in root_xml.findall('juso'):
#         print('-'*100)
#         print('[' + child.findtext('zipNo') + '] ' + child.findtext('roadAddr'))
#         print('    지번주소     = ' + child.findtext('jibunAddr'))
#         print('    영문주소     = ' + child.findtext('engAddr'))
#         print('    도로명코드   = ' + child.findtext('rnMgtSn'))
#         print('    건물관리번호 = ' + child.findtext('bdMgtSn'))
#         print('    법정동코드   = ' + child.findtext('admCd'))
#         print('    상세건물명   = ' + child.findtext('detBdNmList'))
#         print('')

In [179]:
import requests

searching = '김포 현대 프리미엄 아울렛'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
headers = {
    "Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"
}
places = requests.get(url, headers = headers).json()['documents']
places


[{'address_name': '경기 김포시 고촌읍 전호리 654',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '가정,생활 > 상설할인매장',
  'distance': '',
  'id': '22484033',
  'phone': '031-812-2233',
  'place_name': '현대프리미엄아울렛 김포점',
  'place_url': 'http://place.map.kakao.com/22484033',
  'road_address_name': '경기 김포시 고촌읍 아라육로152번길 100',
  'x': '126.785578317669',
  'y': '37.5993284079262'},
 {'address_name': '경기 김포시 고촌읍 전호리 668',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '가정,생활 > 상설할인매장',
  'distance': '',
  'id': '1318163897',
  'phone': '',
  'place_name': '현대프리미엄아울렛 김포점 타워존',
  'place_url': 'http://place.map.kakao.com/1318163897',
  'road_address_name': '경기 김포시 고촌읍 아라육로152번길 97',
  'x': '126.783924573963',
  'y': '37.5979162444969'},
 {'address_name': '경기 김포시 고촌읍 전호리 654',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '가정,생활 > 패션 > 의류판매',
  'distance': '',
  'id': '1384148155',
  'phone': '031-8048-2731',
  'place_name